In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# stuff needed for google colab
CURR_DIR = "/content/drive/My Drive/Grad School Fall/CS 573/Project/colab_notebooks"
DATA_FILE = "profiles.csv"

In [3]:
import pandas as pd
import numpy as np
import os
import nltk
import sys
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from scipy import sparse

np.set_printoptions(threshold=1000)

In [4]:
profiles = pd.read_csv(os.path.join(CURR_DIR, DATA_FILE))
print(profiles)

       age  ...     status
0       22  ...     single
1       35  ...     single
2       38  ...  available
3       23  ...     single
4       29  ...     single
...    ...  ...        ...
59941   59  ...     single
59942   24  ...     single
59943   42  ...     single
59944   27  ...     single
59945   39  ...     single

[59946 rows x 31 columns]


In [5]:
train_df = profiles.sample(10000, random_state=42)

In [19]:
def parseEssay(df):
  essay_list = df['essay0'].to_list()

  essay_list1 = df['essay1'].to_list()
  essay_list2 = df['essay2'].to_list()

  # parses out the NaN's and leaves them empty
  # jesus christ look at this garbage code
  print("parsing out nans")
  for i in range(len(essay_list)):
    if not isinstance(essay_list[i], str):
      if not isinstance(essay_list1[i], str):
        if not isinstance(essay_list1[i], str):
          essay_list[i] = ""
        else:
          essay_list[i] = essay_list2[i]
      else:
        essay_list[i] = essay_list1[i]

  return essay_list

MAX_WORDS = 29000
MAX_LEN = 4000
def createWordVectors(train_essays, max_words, max_len):
  print("encoding essays")
  tok = Tokenizer(num_words=max_words)
  tok.fit_on_texts(train_essays)
  sequences = tok.texts_to_sequences(train_essays)
  sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

  # word_vecs = vectorizer.transform(train_essays)

  return sequences_matrix

train_essays = parseEssay(train_df)
# word_vecs, vectorizer = createWordVectors(train_essays)
word_vecs = createWordVectors(train_essays, MAX_WORDS, MAX_LEN)

X = word_vecs

# for printing out the actual vocab encoding vec in case you want it
# print(vectorizer.vocabulary_)
# encoding document
print(X.shape)


parsing out nans
encoding essays
(10000, 4000)


In [20]:
print(X[0])

[  0   0   0 ... 131 369 514]


In [21]:
def load_labels(df):
  df['sex'].mask(df['sex'] == 'm', 0, inplace=True)
  df['sex'].mask(df['sex'] == 'f', 1, inplace=True)

  labels = df['sex'].to_numpy()

  return labels

labels = load_labels(train_df)

# Convert to float32's so that they play nicely

In [22]:
X = np.asarray(X).astype('float32')
Y = np.asarray(labels).astype('float32')

# Do some neural net shenannigans

In [23]:
from keras.models import Sequential, Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Conv1D, Concatenate
from keras.optimizers import RMSprop, Adam
from sklearn.model_selection import KFold
import statistics 

In [24]:
def createModel(max_words, max_len):
  # max_len = 
  # max_words = 1000
  # inputs = Input(name='inputs',shape=[max_len])
  # layer = Embedding(max_words,50,input_length=max_len)(inputs)
  # layer = LSTM(64)(layer)
  # layer = Dense(256,name='FC1')(layer)
  # layer = Activation('relu')(layer)
  # layer = Dropout(0.5)(layer)
  # layer = Dense(1,name='out_layer')(layer)
  # layer = Activation('sigmoid')(layer)
  # model = Model(inputs=inputs,outputs=layer)

  # model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

  text_input_layer = Input(shape=(max_len,), dtype='int32')
  doc_embedding = Embedding(max_words, 64)(text_input_layer)

  convs = []
  filter_sizes = [10, 20, 30, 40, 50]

  for filter_size in filter_sizes:
    l_conv = Conv1D(filters=64, kernel_size=filter_size, padding='valid', activation='relu')(doc_embedding)
    convs.append(l_conv)

  cnn_feature_maps = Concatenate(axis=1)(convs)

  sentence_encoder = LSTM(64,return_sequences=False)(cnn_feature_maps)
  fc_layer = Dense(128, activation="relu")(sentence_encoder)
  output_layer = Dense(1,activation="sigmoid")(fc_layer)

  model = Model(inputs=[text_input_layer], outputs=[output_layer])

  model.compile(loss='binary_crossentropy',
              optimizer=Adam(1e-4),
              metrics=['accuracy'])

  

  return model

In [25]:
n_split = 10

best_acc = 0
best_model = None

accuracies = []

for train_index,test_index in KFold(n_split).split(X):
  x_train,x_test = X[train_index],X[test_index]
  y_train,y_test = Y[train_index],Y[test_index]

  # print(x_train.shape)
  # print(y_train.shape)
  model = createModel(MAX_WORDS, MAX_LEN)

  model.fit(x_train, y_train,epochs=20)
  
  _, test_acc = model.evaluate(x_test,y_test)
  
  accuracies.append(test_acc)

  # break

avg_acc = statistics.mean(accuracies)
print('Test set accuracy: %.2f' % avg_acc)

Epoch 1/20
282/282 [==============================] - 184s 652ms/step - loss: 0.6788 - accuracy: 0.5887
Epoch 2/20
282/282 [==============================] - 182s 644ms/step - loss: 0.6620 - accuracy: 0.5971
Epoch 3/20
282/282 [==============================] - 184s 653ms/step - loss: 0.5687 - accuracy: 0.7076
Epoch 4/20
282/282 [==============================] - 185s 656ms/step - loss: 0.4217 - accuracy: 0.7929
Epoch 5/20
282/282 [==============================] - 185s 657ms/step - loss: 0.3064 - accuracy: 0.8539
Epoch 6/20
282/282 [==============================] - 188s 667ms/step - loss: 0.2261 - accuracy: 0.8897
Epoch 7/20
282/282 [==============================] - 189s 670ms/step - loss: 0.1737 - accuracy: 0.9134
Epoch 8/20
282/282 [==============================] - 189s 670ms/step - loss: 0.1509 - accuracy: 0.9210
Epoch 9/20
282/282 [==============================] - 189s 670ms/step - loss: 0.1299 - accuracy: 0.9280
Epoch 10/20
282/282 [==============================] - 189s 669m

In [1]:
thingies = [0.6150, 0.6100, 0.6150, 0.5970, 0.6060, 0.5960, 0.5880, 0.6000, 0.6130, 0.5730]

NameError: ignored